### Goal:
#### - Analyzing the impact of Neighborhood Characteristic to Covid19 spread
#### - Verifying social distancing necessity in reducing the number of infection
*Data as of 22nd April 2020

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pandas import json_normalize

from geopy.geocoders import Nominatim

import matplotlib

from sklearn.cluster import KMeans

import folium

### Analyzing Covid 19 data in Toronto
*source: https://www.toronto.ca/home/covid-19/covid-19-latest-city-of-toronto-news/covid-19-status-of-cases-in-toronto/

Retrieving the number of cases per day

In [2]:
#!conda install -c anaconda xlrd

covfile = r'CityofToronto_COVID-19_Data.xlsx'
cov_tor = pd.read_excel(covfile, sheet_name='Cases by Episode Date')
cov_tor.head(60)

,Episode Date,Case Count
0,2020-04-21,2
1,2020-04-20,28
2,2020-04-19,20
3,2020-04-18,36
4,2020-04-17,86
5,2020-04-16,149
6,2020-04-15,190
7,2020-04-14,214
8,2020-04-13,317
9,2020-04-12,128


In [3]:
start = cov_tor['Episode Date'].min()
last = cov_tor['Episode Date'].max()
num = last - start
print('This data is from {} to {}, {} days'.format(start,last,num))


This data is from 2020-01-21 00:00:00 to 2020-04-21 00:00:00, 91 days 00:00:00 days


#### Specifying lockdown time in Ontario
*source: https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Ontario

In [4]:
ON_lockdown_1 = pd.to_datetime('2020-03-17')
ON_lockdown_2 = pd.to_datetime('2020-03-28')

#### Analyzing Covid19 status in Ontario
Source: 
- https://data.ontario.ca/dataset/status-of-covid-19-cases-in-ontario
- https://www.ontario.ca/page/2019-novel-coronavirus#section-4
- https://files.ontario.ca/moh-covid-19-report-en-2020-04-22.pdf
- https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200422-sitrep-93-covid-19.pdf?sfvrsn=35cf80d7_4

In [5]:
cov_on_file = r'covidtesting.csv'
cov_on = pd.read_csv(cov_on_file)
cov_on.head()

,Reported Date,Confirmed Negative,Presumptive Negative,Presumptive Positive,Confirmed Positive,Resolved,Deaths,Total Cases,Total patients approved for testing as of Reporting Date,Total tests completed in the last day,Under Investigation,Number of patients hospitalized with COVID-19,Number of patients in ICU with COVID-19,Number of patients in ICU on a ventilator with COVID-19
0,2020-01-26,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN
1,2020-01-27,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN
2,2020-01-28,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,11,NaN,NaN,NaN
3,2020-01-30,NaN,NaN,0.0,2.0,NaN,NaN,2.0,NaN,NaN,27,NaN,NaN,NaN
4,2020-01-31,NaN,NaN,0.0,2.0,NaN,NaN,2.0,NaN,NaN,17,NaN,NaN,NaN


In [10]:
cov_on.columns

Index(['Reported Date', 'Confirmed Negative', 'Presumptive Negative',
       'Presumptive Positive', 'Confirmed Positive', 'Resolved', 'Deaths',
       'Total Cases',
       'Total patients approved for testing as of Reporting Date',
       'Total tests completed in the last day', 'Under Investigation',
       'Number of patients hospitalized with COVID-19',
       'Number of patients in ICU with COVID-19',
       'Number of patients in ICU on a ventilator with COVID-19'],
      dtype='object')

#### Analyzing confirmed positive cases in Ontario
*source: https://data.ontario.ca/dataset/confirmed-positive-cases-of-covid-19-in-ontario

In [7]:
cov_pos_file = r'conposcovidloc.csv'
cov_pos = pd.read_csv(cov_pos_file)
cov_pos.head()

,Row_ID,Accurate_Episode_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,1,2020-01-22,50s,FEMALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
1,2,2020-01-21,50s,MALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
2,3,2020-01-24,20s,FEMALE,Travel-Related,Resolved,Middlesex-London Health Unit,50 King Street,London,N6A 5L7,www.healthunit.com,42.981468,-81.254016
3,4,2020-02-05,20s,FEMALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
4,5,2020-02-16,60s,FEMALE,Travel-Related,Resolved,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358


In [8]:
cov_pos.shape

(12245, 13)